In [5]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

protoFile = "hand/pose_deploy.prototxt"
weightsFile = "hand/pose_iter_102000.caffemodel"
nPoints = 22
POSE_PAIRS = [ [0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20] ]
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [6]:
import os
from os import listdir
from os.path import isfile, join

In [7]:
mypath = 'dorsal_right'
image_files = [ f for f in listdir(mypath) if isfile(join(mypath,f))]

In [4]:
image_files

['Hand_0000021.jpg.jpg',
 'Hand_0000022.jpg.jpg',
 'Hand_0000023.jpg.jpg',
 'Hand_0000093.jpg',
 'Hand_0000094.jpg',
 'Hand_0000159.jpg.jpg',
 'Hand_0000197.jpg.jpg',
 'Hand_0000219.jpg.jpg',
 'Hand_0000220.jpg.jpg',
 'Hand_0000221.jpg.jpg',
 'Hand_0001074.jpg.jpg',
 'Hand_0001075.jpg.jpg',
 'Hand_0001076.jpg.jpg',
 'Hand_0001077.jpg.jpg',
 'Hand_0001551.jpg.jpg',
 'Hand_0001552.jpg.jpg',
 'Hand_0001702.jpg.jpg',
 'Hand_0001703.jpg.jpg',
 'Hand_0001949.jpg.jpg',
 'Hand_0001950.jpg.jpg',
 'Hand_0002264.jpg.jpg',
 'Hand_0002265.jpg.jpg',
 'Hand_0002784.jpg.jpg',
 'Hand_0002785.jpg.jpg',
 'Hand_0004725.jpg.jpg',
 'Hand_0004726.jpg.jpg',
 'Hand_0004892.jpg.jpg',
 'Hand_0004893.jpg.jpg',
 'Hand_0004894.jpg.jpg',
 'Hand_0004895.jpg.jpg',
 'Hand_0005040.jpg.jpg',
 'Hand_0005041.jpg.jpg',
 'Hand_0006090.jpg.jpg',
 'Hand_0006091.jpg.jpg',
 'Hand_0006092.jpg.jpg',
 'Hand_0006776.jpg.jpg',
 'Hand_0006777.jpg.jpg',
 'Hand_0006778.jpg.jpg',
 'Hand_0006779.jpg.jpg',
 'Hand_0009068.jpg.jpg',
 'Hand_0

In [8]:
for i in range(len(image_files)):
    print(image_files[i])

Hand_0000021.jpg.jpg
Hand_0000022.jpg.jpg
Hand_0000023.jpg.jpg
Hand_0000093.jpg
Hand_0000094.jpg
Hand_0000159.jpg.jpg
Hand_0000197.jpg.jpg
Hand_0000219.jpg.jpg
Hand_0000220.jpg.jpg
Hand_0000221.jpg.jpg
Hand_0001074.jpg.jpg
Hand_0001075.jpg.jpg
Hand_0001076.jpg.jpg
Hand_0001077.jpg.jpg
Hand_0001551.jpg.jpg
Hand_0001552.jpg.jpg
Hand_0001702.jpg.jpg
Hand_0001703.jpg.jpg
Hand_0001949.jpg.jpg
Hand_0001950.jpg.jpg
Hand_0002264.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002784.jpg.jpg
Hand_0002785.jpg.jpg
Hand_0004725.jpg.jpg
Hand_0004726.jpg.jpg
Hand_0004892.jpg.jpg
Hand_0004893.jpg.jpg
Hand_0004894.jpg.jpg
Hand_0004895.jpg.jpg
Hand_0005040.jpg.jpg
Hand_0005041.jpg.jpg
Hand_0006090.jpg.jpg
Hand_0006091.jpg.jpg
Hand_0006092.jpg.jpg
Hand_0006776.jpg.jpg
Hand_0006777.jpg.jpg
Hand_0006778.jpg.jpg
Hand_0006779.jpg.jpg
Hand_0009068.jpg.jpg
Hand_0009069.jpg.jpg
Hand_0009190.jpg.jpg
Hand_0009191.jpg.jpg
Hand_0009357.jpg.jpg
Hand_0009358.jpg.jpg
Hand_0009519.jpg.jpg
Hand_0009520.jpg.jpg
Hand_0009936.jpg.jpg


In [15]:
for i in range(len(image_files)):
    frame = cv2.imread(mypath + '/' + image_files[i])
    tmp = image_files[i]
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    aspect_ratio = frameWidth/frameHeight
    threshold = 0.1

    t = time.time()
    # input image dimensions for the network
    inHeight = 368
    # inWidth = 368
    inWidth = int(((aspect_ratio*inHeight)*8)//8)
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    print("time taken by network : {:.3f}".format(time.time() - t))
    # Empty list to store the detected keypoints
    points = []
    print(image_files[i])#
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            points.append(None)
    print(image_files[i])#
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.imwrite('./result/dorsal_left/'+ tmp, frame)
    print(image_files[i])#
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    


    print("Total time taken : {:.3f}".format(time.time() - t))

time taken by network : 1.513
Hand_0000021.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 1.601
time taken by network : 1.487
Hand_0000022.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 1.572
time taken by network : 1.608
Hand_0000023.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 1.768
time taken by network : 2.852
Hand_0000093.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 3.018
time taken by network : 2.502
Hand_0000094.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.678
time taken by network : 2.694
Hand_0000159.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.872
time taken by network : 3.238
Hand_0000197.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 3.441
time taken by network : 2.816
Hand_0000219.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.978
time taken by network : 2.767
Hand_0000220.jpg.jpg
Hand_0002265.

time taken by network : 2.523
Hand_0011723.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.730
time taken by network : 2.548
Hand_0011724.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.746
time taken by network : 2.632
Hand_0011727.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.816
time taken by network : 2.610
Hand_0011728.jpg.jpg
Hand_0002265.jpg.jpg
Hand_0002265.jpg.jpg
Total time taken : 2.792


In [18]:
mypath = 'dorsal_right'
image_files = [ f for f in listdir(mypath) if isfile(join(mypath,f))]
for i in range(len(image_files)):
    frame = cv2.imread(mypath + '/' + image_files[i])
    tmp = image_files[i]
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    aspect_ratio = frameWidth/frameHeight
    threshold = 0.1

    t = time.time()
    # input image dimensions for the network
    inHeight = 368
    # inWidth = 368
    inWidth = int(((aspect_ratio*inHeight)*8)//8)
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    print("time taken by network : {:.3f}".format(time.time() - t))
    # Empty list to store the detected keypoints
    points = []
    print(image_files[i])#
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            points.append(None)
    print(image_files[i])#
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.imwrite('./result/dorsal_right/'+ tmp, frame)
    print(image_files[i])#
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    


    print("Total time taken : {:.3f}".format(time.time() - t))

time taken by network : 1.915
Hand_0000002.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 2.093
time taken by network : 2.861
Hand_0000003.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 3.048
time taken by network : 4.021
Hand_0000014.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 4.226
time taken by network : 2.945
Hand_0000015.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 3.151
time taken by network : 2.646
Hand_0000016.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 2.813
time taken by network : 2.621
Hand_0000265.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 2.825
time taken by network : 2.595
Hand_0000396.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 2.777
time taken by network : 2.647
Hand_0000427.jpg.jpg
Hand_0002319.jpg.jpg
Hand_0002319.jpg.jpg
Total time taken : 2.817
time taken by network : 2.736
Hand_0000428.jpg.jpg
Hand_

In [19]:
mypath = 'palmar_left'
image_files = [ f for f in listdir(mypath) if isfile(join(mypath,f))]
for i in range(len(image_files)):
    frame = cv2.imread(mypath + '/' + image_files[i])
    tmp = image_files[i]
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    aspect_ratio = frameWidth/frameHeight
    threshold = 0.1

    t = time.time()
    # input image dimensions for the network
    inHeight = 368
    # inWidth = 368
    inWidth = int(((aspect_ratio*inHeight)*8)//8)
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    print("time taken by network : {:.3f}".format(time.time() - t))
    # Empty list to store the detected keypoints
    points = []
    print(image_files[i])#
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            points.append(None)
    print(image_files[i])#
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.imwrite('./result/palmar_left/'+ tmp, frame)
    print(image_files[i])#
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    


    print("Total time taken : {:.3f}".format(time.time() - t))

time taken by network : 2.742
Hand_0000215.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.936
time taken by network : 2.569
Hand_0000383.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.741
time taken by network : 2.540
Hand_0000491.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.708
time taken by network : 2.577
Hand_0000583.jpg.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.756
time taken by network : 2.603
Hand_0000584.jpg.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.786
time taken by network : 2.544
Hand_0000745.jpg.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.728
time taken by network : 2.548
Hand_0000746.jpg.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.712
time taken by network : 2.580
Hand_0000748.jpg.jpg
Hand_0002038.jpg.jpg
Hand_0002038.jpg.jpg
Total time taken : 2.752
time taken by network : 2.528
Hand_0000754.jpg.jpg
Hand_0002038.jpg.

In [20]:
mypath = 'palmar_right'
image_files = [ f for f in listdir(mypath) if isfile(join(mypath,f))]
for i in range(len(image_files)):
    frame = cv2.imread(mypath + '/' + image_files[i])
    tmp = image_files[i]
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    aspect_ratio = frameWidth/frameHeight
    threshold = 0.1

    t = time.time()
    # input image dimensions for the network
    inHeight = 368
    # inWidth = 368
    inWidth = int(((aspect_ratio*inHeight)*8)//8)
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    print("time taken by network : {:.3f}".format(time.time() - t))
    # Empty list to store the detected keypoints
    points = []
    print(image_files[i])#
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            points.append(None)
    print(image_files[i])#
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.imwrite('./result/palmar_right/'+ tmp, frame)
    print(image_files[i])#
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
#     plt.figure(figsize=[10,10])
#     plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    


    print("Total time taken : {:.3f}".format(time.time() - t))

time taken by network : 2.700
Hand_0000187.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.880
time taken by network : 2.550
Hand_0000232.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.726
time taken by network : 2.645
Hand_0000233.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.827
time taken by network : 2.544
Hand_0001152.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.714
time taken by network : 2.511
Hand_0001153.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.683
time taken by network : 2.605
Hand_0001389.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.790
time taken by network : 2.635
Hand_0001390.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.804
time taken by network : 2.580
Hand_0001429.jpg.jpg
Hand_0004582.jpg.jpg
Hand_0004582.jpg.jpg
Total time taken : 2.744
time taken by network : 2.604
Hand_0001430.jpg.jpg
Hand_